**Loading Data, see the VIF to find Multicollinearity**

In [103]:
import pandas as pd
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm

X_train = pd.read_csv('../data/X_train.csv')
y_train = pd.read_csv('../data/y_train.csv')
X_test = pd.read_csv('../data/X_test.csv')
y_test = pd.read_csv('../data/y_test.csv')

# Convert it to a Series for Statsmodels.
y_train = y_train.squeeze()
y_test = y_test.squeeze()


# List of reference categories to drop
# (We drop 'Bus' and 'Night' so they become the standard baseline)
cols_to_drop = ['Vehicle_Bus','TimeOfDay_Night']


X_train = X_train.drop(columns=cols_to_drop, errors='ignore')
X_test = X_test.drop(columns=cols_to_drop, errors='ignore')

print("Reference categories dropped. Ready for VIF check or Modeling.")

# Create a VIF dataframe
vif_data = pd.DataFrame()
vif_data["feature"] = X_train.columns
vif_data["VIF"] = [variance_inflation_factor(X_train.values, i) 
                   for i in range(len(X_train.columns))]

print(vif_data.sort_values(by="VIF", ascending=False))

Reference categories dropped. Ready for VIF check or Modeling.
                feature         VIF
2              LogPrice  132.674476
0              Domestic   89.891337
10              To_Rate   43.458594
9             From_Rate   42.755284
11           Route_Rate   21.274792
12            User_Rate    7.730354
3           LogLeadTime    4.950620
6   TimeOfDay_Afternoon    3.135194
7     TimeOfDay_Evening    2.753429
1            TripReason    2.699448
5         Vehicle_Train    2.430876
8     TimeOfDay_Morning    2.203497
4         Vehicle_Plane    1.912165


In [104]:
# 1. Drop the redundant location parts (Keep only Route)
cols_to_drop = ['From_Rate', 'To_Rate', 'Domestic']

# 2. Drop Price (Because it is highly collinear with Route + Vehicle)
cols_to_drop.append('LogPrice')

# Apply the drops
X_train = X_train.drop(columns=cols_to_drop, errors='ignore')
X_test = X_test.drop(columns=cols_to_drop, errors='ignore')

# 3. Check VIF
vif_data = pd.DataFrame()
vif_data["feature"] = X_train.columns
vif_data["VIF"] = [variance_inflation_factor(X_train.values, i) 
                   for i in range(len(X_train.columns))]

print(vif_data.sort_values(by="VIF", ascending=False))

               feature       VIF
7           Route_Rate  8.017524
8            User_Rate  6.427685
1          LogLeadTime  4.575417
4  TimeOfDay_Afternoon  2.550808
3        Vehicle_Train  2.360378
0           TripReason  2.304457
5    TimeOfDay_Evening  2.297533
6    TimeOfDay_Morning  1.866953
2        Vehicle_Plane  1.459764


## Class Imbalance Problem & Solution

**Problem:** Our data has 13.7% positive class (cancellations) vs 86.3% negative (no cancellation).
- Model learns to predict 0 (non-cancellation) always → High accuracy, but useless!
- Precision/Recall become 0 because model never predicts 1.

**Solutions:**
**Oversampling (SMOTE)** - Create synthetic samples of minority class

## Scaler

**Problem** High OR in Route_Rate and User_Rate

**Solution**
We used standardScaler to balanced data to have a data in same range

In [105]:
# === STEP 1: SMOTE + SCALING ===
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler

print("="*70)
print("STEP 1: APPLY SMOTE TO TRAINING DATA ONLY")
print("="*70)

print("\nBEFORE Oversampling (Training Data):")
print(f"Class 0 (No Cancel): {(y_train == 0).sum()} samples")
print(f"Class 1 (Cancel):    {(y_train == 1).sum()} samples")
print(f"Ratio: {(y_train == 1).sum() / len(y_train) * 100:.2f}% cancellations")

# Apply SMOTE ONLY to training data
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

print("\nAFTER Oversampling (Training Data):")
print(f"Class 0 (No Cancel): {(y_train_balanced == 0).sum()} samples")
print(f"Class 1 (Cancel):    {(y_train_balanced == 1).sum()} samples")
print(f"Ratio: {(y_train_balanced == 1).sum() / len(y_train_balanced) * 100:.2f}% cancellations")

print("\n" + "="*70)
print("STEP 2: APPLY STANDARD SCALING")
print("="*70)

# Fit scaler on balanced training data, transform both train and test
scaler = StandardScaler()
X_train_balanced_scaled = pd.DataFrame(
    scaler.fit_transform(X_train_balanced), 
    columns=X_train_balanced.columns
)
X_test_scaled = pd.DataFrame(
    scaler.transform(X_test), 
    columns=X_test.columns
)

print("Scaling applied successfully!")
print(f"X_train_balanced_scaled shape: {X_train_balanced_scaled.shape}")
print(f"X_test_scaled shape: {X_test_scaled.shape}")

STEP 1: APPLY SMOTE TO TRAINING DATA ONLY

BEFORE Oversampling (Training Data):
Class 0 (No Cancel): 68553 samples
Class 1 (Cancel):    10911 samples
Ratio: 13.73% cancellations

AFTER Oversampling (Training Data):
Class 0 (No Cancel): 68553 samples
Class 1 (Cancel):    68553 samples
Ratio: 50.00% cancellations

STEP 2: APPLY STANDARD SCALING
Scaling applied successfully!
X_train_balanced_scaled shape: (137106, 9)
X_test_scaled shape: (19866, 9)


In [106]:
# === STEP 3: BASELINE MODEL (LogLeadTime Only) ===

print("="*70)
print("STEP 3: BASELINE MODEL - LogLeadTime Only")
print("="*70)

# Prepare baseline data (LogLeadTime only) from scaled data
X_train_balanced_scaled_baseline = X_train_balanced_scaled[['LogLeadTime']].copy()
X_train_balanced_scaled_baseline_const = sm.add_constant(X_train_balanced_scaled_baseline)

X_test_scaled_baseline = X_test_scaled[['LogLeadTime']].copy()
X_test_scaled_baseline_const = sm.add_constant(X_test_scaled_baseline)

# Fit baseline model
baseline_model = sm.GLM(y_train_balanced, X_train_balanced_scaled_baseline_const, 
                        family=sm.families.Binomial()).fit()

print(baseline_model.summary())

# Predictions using threshold = 0.5
threshold = 0.5
y_pred_prob_baseline = baseline_model.predict(X_test_scaled_baseline_const)
y_pred_baseline = (y_pred_prob_baseline >= threshold).astype(int)

print(f"\nAIC: {baseline_model.aic:.2f}")
print(f"Log-Likelihood: {baseline_model.llf:.2f}")

STEP 3: BASELINE MODEL - LogLeadTime Only
                 Generalized Linear Model Regression Results                  
Dep. Variable:                 Cancel   No. Observations:               137106
Model:                            GLM   Df Residuals:                   137104
Model Family:                Binomial   Df Model:                            1
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -92419.
Date:                Mon, 26 Jan 2026   Deviance:                   1.8484e+05
Time:                        11:18:21   Pearson chi2:                 1.37e+05
No. Iterations:                     4   Pseudo R-squ. (CS):            0.03743
Covariance Type:            nonrobust                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
const   

In [107]:
# === STEP 4: FULL MODEL WITH ALL FEATURES ===
from scipy import stats
from sklearn.metrics import log_loss, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np

print("="*70)
print("STEP 4: FULL MODEL - ALL FEATURES")
print("="*70)

# Add constant to scaled data
X_train_balanced_scaled_const = sm.add_constant(X_train_balanced_scaled)
X_test_scaled_const = sm.add_constant(X_test_scaled)

# Fit full model on balanced and scaled training data
full_model = sm.GLM(y_train_balanced, X_train_balanced_scaled_const, 
                    family=sm.families.Binomial()).fit()

print(full_model.summary())

# Predictions using threshold = 0.5
threshold = 0.5
y_pred_prob_full = full_model.predict(X_test_scaled_const)
y_pred_full = (y_pred_prob_full >= threshold).astype(int)

print("\n" + "="*70)
print("MODEL COMPARISON: BASELINE vs FULL MODEL")
print("="*70)

# Cross-Entropy Loss
baseline_cross_entropy = log_loss(y_test, y_pred_prob_baseline)
full_cross_entropy = log_loss(y_test, y_pred_prob_full)

# Create comparison dataframe
comparison_data = {
    'Metric': ['AIC', 'BIC', 'Log-Likelihood', 'Cross-Entropy Loss', 'Features Used'],
    'Baseline (LogLeadTime)': [
        f"{baseline_model.aic:.2f}",
        f"{baseline_model.bic:.2f}",
        f"{baseline_model.llf:.2f}",
        f"{baseline_cross_entropy:.4f}",
        "1"
    ],
    'Full Model (All)': [
        f"{full_model.aic:.2f}",
        f"{full_model.bic:.2f}",
        f"{full_model.llf:.2f}",
        f"{full_cross_entropy:.4f}",
        f"{len(X_train_balanced_scaled.columns)}"
    ]
}

comparison_df = pd.DataFrame(comparison_data)
print(comparison_df.to_string(index=False))


# Performance comparison
print(f"{'Metric':<15} {'Baseline':<15} {'Full Model':<15}")
print("-"*45)

baseline_acc = accuracy_score(y_test, y_pred_baseline)
full_acc = accuracy_score(y_test, y_pred_full)
print(f"{'Accuracy':<15} {baseline_acc:<15.4f} {full_acc:<15.4f}")

baseline_prec = precision_score(y_test, y_pred_baseline, zero_division=0)
full_prec = precision_score(y_test, y_pred_full, zero_division=0)
print(f"{'Precision':<15} {baseline_prec:<15.4f} {full_prec:<15.4f}")

baseline_rec = recall_score(y_test, y_pred_baseline, zero_division=0)
full_rec = recall_score(y_test, y_pred_full, zero_division=0)
print(f"{'Recall':<15} {baseline_rec:<15.4f} {full_rec:<15.4f}")

baseline_f1 = f1_score(y_test, y_pred_baseline, zero_division=0)
full_f1 = f1_score(y_test, y_pred_full, zero_division=0)
print(f"{'F1-Score':<15} {baseline_f1:<15.4f} {full_f1:<15.4f}")

baseline_auc = roc_auc_score(y_test, y_pred_prob_baseline)
full_auc = roc_auc_score(y_test, y_pred_prob_full)
print(f"{'ROC-AUC':<15} {baseline_auc:<15.4f} {full_auc:<15.4f}")

STEP 4: FULL MODEL - ALL FEATURES
                 Generalized Linear Model Regression Results                  
Dep. Variable:                 Cancel   No. Observations:               137106
Model:                            GLM   Df Residuals:                   137096
Model Family:                Binomial   Df Model:                            9
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -88480.
Date:                Mon, 26 Jan 2026   Deviance:                   1.7696e+05
Time:                        11:18:21   Pearson chi2:                 1.37e+05
No. Iterations:                     5   Pseudo R-squ. (CS):            0.09119
Covariance Type:            nonrobust                                         
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------


c:\Users\win_10\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\genmod\generalized_linear_model.py:1923: FutureWarning: The bic value is computed using the deviance formula. After 0.13 this will change to the log-likelihood based formula. This change has no impact on the relative rank of models compared using BIC. You can directly access the log-likelihood version using the `bic_llf` attribute. You can suppress this message by calling statsmodels.genmod.generalized_linear_model.SET_USE_BIC_LLF with True to get the LLF-based version now or False to retainthe deviance version.
  warnings.warn(


In [108]:
# === STEP 5: FEATURE IMPORTANCE & ODDS RATIOS ===

print("="*70)
print("STEP 5: FEATURE IMPORTANCE (Odds Ratios) - Scaled Data")
print("="*70)


# Extract coefficients and calculate odds ratios
coef_summary = pd.DataFrame({
    'Feature': X_train_balanced_scaled_const.columns,
    'Coefficient': full_model.params.values,
    'Std Error': full_model.bse.values,
    'P-value': full_model.pvalues.values
})

# Calculate Odds Ratio = exp(coefficient)
coef_summary['Odds Ratio'] = np.exp(coef_summary['Coefficient'])
coef_summary['OR Interpretation'] = coef_summary['Odds Ratio'].apply(
    lambda x: f"{'↑' if x > 1 else '↓'} {abs((x-1)*100):.1f}% per 1 SD" if x != 1 else "No effect"
)

# Sort by absolute coefficient value
coef_summary['Abs_Coef'] = abs(coef_summary['Coefficient'])
coef_summary_sorted = coef_summary.sort_values('Abs_Coef', ascending=False).drop('Abs_Coef', axis=1)

print(coef_summary_sorted.to_string(index=False))

# Show significant features
print("\nSignificant Features (p < 0.05):")
significant = coef_summary_sorted[coef_summary_sorted['P-value'] < 0.05]
for idx, row in significant.iterrows(): 
    effect = "INCREASES" if row['Coefficient'] > 0 else "DECREASES"
    print(f"  ✓ {row['Feature']:30} → {effect:10} (p={row['P-value']:.4f}, OR={row['Odds Ratio']:.3f})")

print("\nInsignificant Features (p ≥ 0.05):")
insignificant = coef_summary_sorted[coef_summary_sorted['P-value'] >= 0.05]
for idx, row in insignificant.iterrows():  
    print(f"  ✗ {row['Feature']:30} (p={row['P-value']:.4f})")

STEP 5: FEATURE IMPORTANCE (Odds Ratios) - Scaled Data
            Feature  Coefficient  Std Error       P-value  Odds Ratio OR Interpretation
        LogLeadTime     0.435268   0.006954  0.000000e+00    1.545378  ↑ 54.5% per 1 SD
          User_Rate     0.346990   0.006395  0.000000e+00    1.414803  ↑ 41.5% per 1 SD
         Route_Rate     0.311553   0.006137  0.000000e+00    1.365544  ↑ 36.6% per 1 SD
      Vehicle_Plane    -0.165968   0.006358 3.252737e-150    0.847073  ↓ 15.3% per 1 SD
         TripReason     0.129932   0.006211  3.599518e-97    1.138750  ↑ 13.9% per 1 SD
      Vehicle_Train    -0.093870   0.006985  3.588965e-41    0.910401   ↓ 9.0% per 1 SD
  TimeOfDay_Evening    -0.055191   0.007785  1.343343e-12    0.946304   ↓ 5.4% per 1 SD
TimeOfDay_Afternoon    -0.025518   0.007992  1.408579e-03    0.974805   ↓ 2.5% per 1 SD
  TimeOfDay_Morning    -0.021293   0.007470  4.367308e-03    0.978932   ↓ 2.1% per 1 SD
              const     0.010057   0.005681  7.667994e-02    1.01